In [7]:
import cvxpy as cp
import numpy as np

# Variables
x1 = cp.Variable((1, 24), boolean=True)  # Bronny
x2 = cp.Variable((1, 24), boolean=True)  # Bryce
x3 = cp.Variable((1, 24), boolean=True)  # Zhuri

# Parameters
worth = np.array([[30], [30], [300], [90], [28], [10], [23], [36.75],
                  [101.355], [0.1], [1], [-0.575244], [0.428], [-2], [0.5], [0.335],
                  [0.312], [1], [0.5], [0.22], [0.206], [0.26], [0.31], [0.19]])
satisfaction = np.array([
    [11, 5, 16], [17, 6, 2], [12, 1, 8], [3, 2, 13],
    [10, 10, 18], [9, 7, 17], [1, 4, 9], [8, 3, 1],
    [2, 9, 10], [7, 8, 4], [16, 11, 5], [15, 15, 15],
    [14, 13, 19], [21, 20, 23], [6, 19, 12], [5, 21, 21],
    [22, 22, 14], [20, 23, 22], [19, 14, 20], [13, 18, 3],
    [24, 17, 6], [23, 24, 7], [4, 12, 11], [18, 16, 24]
])

# Satisfaction indices
r1 = satisfaction[:, 0].reshape(-1, 1)
r2 = satisfaction[:, 1].reshape(-1, 1)
r3 = satisfaction[:, 2].reshape(-1, 1)

# Compute averages
average_worth = np.sum(worth)/3

# Constraints
constraints = []

# Ensure each asset is assigned to exactly one person
for i in range(24):
    constraints.append(x1[0, i] + x2[0, i] + x3[0, i] == 1)

# Total worth for each person
worth_total_bronny = x1 @ worth
worth_total_bryce = x2 @ worth
worth_total_zhuri = x3 @ worth

# Balance worth (allow a tolerance of 125)
constraints.append(worth_total_bronny <= worth_total_bryce + 125)
constraints.append(worth_total_bronny >= worth_total_bryce - 125)
constraints.append(worth_total_bryce <= worth_total_zhuri + 125)
constraints.append(worth_total_bryce >= worth_total_zhuri - 125)
constraints.append(worth_total_zhuri <= worth_total_bronny + 125)
constraints.append(worth_total_zhuri >= worth_total_bronny - 125)

# Total satisfaction for each person
satisfaction_bronny = x1 @ r1
satisfaction_bryce = x2 @ r2
satisfaction_zhuri = x3 @ r3

# Balance satisfaction (allow a tolerance of 30)
constraints.append(satisfaction_bronny <= satisfaction_bryce + 30)
constraints.append(satisfaction_bronny >= satisfaction_bryce - 30)
constraints.append(satisfaction_bryce <= satisfaction_zhuri + 30)
constraints.append(satisfaction_bryce >= satisfaction_zhuri - 30)
constraints.append(satisfaction_zhuri <= satisfaction_bronny + 30)
constraints.append(satisfaction_zhuri >= satisfaction_bronny - 30)

# Dummy objective (no actual optimization)
obj_func = cp.Minimize(0)

# Solve the problem
problem = cp.Problem(obj_func, constraints)
problem.solve(solver=cp.GUROBI, verbose=False)

# Print results
print("Solver status:", problem.status)
print("Bronny (x1):")
print(x1.value)
print("Bryce (x2):")
print(x2.value)
print("Zhuri (x3):")
print(x3.value)

print("Average worth: ",average_worth)

print("\nSatisfaction Scores:")
print("Bronny:", satisfaction_bronny.value)
print("Bryce:", satisfaction_bryce.value)
print("Zhuri:", satisfaction_zhuri.value)

print("\nTotal Worth:")
print("Bronny:", worth_total_bronny.value)
print("Bryce:", worth_total_bryce.value)
print("Zhuri:", worth_total_zhuri.value)
print(worth_total_bronny.value+worth_total_bryce.value+worth_total_zhuri.value)


Solver status: optimal
Bronny (x1):
[[ 1.  1. -0. -0. -0.  1. -0. -0.  1. -0. -0. -0. -0. -0.  1.  1.  1.  1.
  -0. -0.  1. -0. -0. -0.]]
Bryce (x2):
[[-0. -0. -0.  1.  1. -0.  1.  1. -0.  1.  1. -0. -0. -0. -0. -0. -0. -0.
  -0. -0. -0.  1.  1.  1.]]
Zhuri (x3):
[[-0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0.  1.  1.  1. -0. -0. -0. -0.
   1.  1. -0. -0. -0. -0.]]
Average worth:  217.29691866666667

Satisfaction Scores:
Bronny: [[116.]]
Bryce: [[90.]]
Zhuri: [[88.]]

Total Worth:
Bronny: [[173.708]]
Bryce: [[179.61]]
Zhuri: [[298.572756]]
[[651.890756]]
